In [8]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
from keras import optimizers
import shutil
import random
from PIL import Image
import numpy as np
import logging
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil
from tensorflow import debugging
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint

from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from keras.regularizers import l2

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

model_path = "./deep_ranking"

# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

2.8.0


# Step 1: create lumo predictor

In [9]:

X = np.array(pd.read_csv("pretrain_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y = np.array(pd.read_csv("pretrain_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

lumo_predictor = tf.keras.Sequential([
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='linear')
        ])
        
lumo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.0001))
history = lumo_predictor.fit(X, Y, epochs=30)

lumo_predictor.save_weights('lumo_predictor.h5')

Epoch 1/30
1563/1563 [==============================] - 6s 3ms/step - loss: 0.0999
Epoch 2/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0107
Epoch 3/30
1563/1563 [==============================] - 5s 3ms/step - loss: 0.0066
Epoch 4/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0047
Epoch 5/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0036
Epoch 6/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0028
Epoch 7/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0024
Epoch 8/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0020
Epoch 9/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0017
Epoch 10/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0015
Epoch 11/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0014
Epoch 12/30
1563/1563 [==============================] - 4s 3ms/step - loss: 0.0013
E

# Step 2: setup data to train homo predictor

In [10]:
X_train = np.array(pd.read_csv("train_features.csv.zip").iloc[:, 2:]).astype(np.float32)
Y_train = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_gap = np.array(pd.read_csv("train_labels.csv.zip").iloc[:, 1:]).astype(np.float32)

Y_lumo = lumo_predictor.predict(X_train)

Y_homo = -(Y_gap - Y_lumo)



# Step 3: train homo predictor

In [11]:
homo_predictor = tf.keras.Sequential([
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            layers.Dense(512, activation=tf.keras.activations.relu, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001)),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            # layers.Dense(512, activation=tf.keras.activations.relu),
            layers.Dense(1, activation='linear')
        ])

#build the model
homo_predictor.predict(X_train)

#transfer learning
homo_predictor.load_weights("lumo_predictor.h5")

homo_predictor.compile(loss=losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(0.00001))
history = homo_predictor.fit(X_train, Y_homo, epochs=450)



Epoch 1/450
4/4 [==============================] - 0s 3ms/step - loss: 3.5075
Epoch 2/450
4/4 [==============================] - 0s 3ms/step - loss: 3.3613
Epoch 3/450
4/4 [==============================] - 0s 4ms/step - loss: 3.2209
Epoch 4/450
4/4 [==============================] - 0s 3ms/step - loss: 3.0840
Epoch 5/450
4/4 [==============================] - 0s 3ms/step - loss: 2.9499
Epoch 6/450
4/4 [==============================] - 0s 3ms/step - loss: 2.8199
Epoch 7/450
4/4 [==============================] - 0s 3ms/step - loss: 2.6918
Epoch 8/450
4/4 [==============================] - 0s 3ms/step - loss: 2.5668
Epoch 9/450
4/4 [==============================] - 0s 3ms/step - loss: 2.4447
Epoch 10/450
4/4 [==============================] - 0s 3ms/step - loss: 2.3253
Epoch 11/450
4/4 [==============================] - 0s 3ms/step - loss: 2.2107
Epoch 12/450
4/4 [==============================] - 0s 3ms/step - loss: 2.0984
Epoch 13/450
4/4 [==============================] - 0s 2ms/st

# Step 4: test set

In [12]:
X_test = np.array(pd.read_csv("test_features.csv.zip").iloc[:, 2:]).astype(np.float32)
sample = pd.read_csv("sample.csv")


homo = homo_predictor.predict(X_test)
lumo = lumo_predictor.predict(X_test)

sample["y"] = (lumo-homo)

sample.to_csv('submit.csv', index=False, float_format='%.5f')